Install the dependencies to your computer and import them

In [1]:
import sys
!{sys.executable} -m pip install fpdf -q
!{sys.executable} -m pip install pandas -q
!{sys.executable} -m pip install plotly-express -q
!{sys.executable} -m pip install kaleido -q

In [2]:
from datetime import date
from pathlib import Path
import sqlite3

import pandas as pd
import plotly.express as px
from fpdf import FPDF

Define the paths and chart style

In [3]:
# Define the plotly template
plotly_template = "presentation"

In [4]:
# Define the paths
current_dir = Path(__file__).parents if "__file__" in locals() else Path.cwd()
db_path = current_dir / "sales.db"
output_dir = current_dir / "output"

# Create the output directory if it doesn't exist
output_dir.mkdir(parents=True, exist_ok=True)

In [5]:
# Connect to the database
conn = sqlite3.connect(db_path)

In [6]:
# Execute the query and load the results into a Pandas dataframe
query = '''
SELECT sale_date, SUM(total_price) as total_sales
FROM sales
GROUP BY sale_date
ORDER BY sale_date ASC
'''
df = pd.read_sql_query(query, conn)

In [7]:
# Print the dataframe
print(df)

      sale_date  total_sales
0    2023-01-01        12985
1    2023-01-02         9225
2    2023-01-03         8985
3    2023-01-04        11600
4    2023-01-05        10560
..          ...          ...
360  2023-12-27         9595
361  2023-12-28        11465
362  2023-12-29        10395
363  2023-12-30        13000
364  2023-12-31        11150

[365 rows x 2 columns]


In [8]:
# Check the data types
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 365 entries, 0 to 364
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   sale_date    365 non-null    object
 1   total_sales  365 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 5.8+ KB


In [9]:
# Convert the sale_date column to a datetime
df['sale_date'] = pd.to_datetime(df['sale_date'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 365 entries, 0 to 364
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   sale_date    365 non-null    datetime64[ns]
 1   total_sales  365 non-null    int64         
dtypes: datetime64[ns](1), int64(1)
memory usage: 5.8 KB


In [10]:
# Set the sale_date column as the index
df = df.set_index('sale_date')
df.head(3)

,total_sales
sale_date,
2023-01-01,12985
2023-01-02,9225
2023-01-03,8985


In [11]:
# Resample the data to calculate sales by month and calculate the sum
df_monthly = df.resample('M').sum()
df_monthly

,total_sales
sale_date,
2023-01-31,353380
2023-02-28,331375
2023-03-31,350865
2023-04-30,351405
2023-05-31,344770
2023-06-30,333745
2023-07-31,354370
2023-08-31,348450
2023-09-30,340615


In [12]:
# Map the month number to the month name to display the month name on the chart
df_monthly['month_name'] = df_monthly.index.strftime('%b')
df_monthly

,total_sales,month_name
sale_date,,
2023-01-31,353380,Jan
2023-02-28,331375,Feb
2023-03-31,350865,Mar
2023-04-30,351405,Apr
2023-05-31,344770,May
2023-06-30,333745,Jun
2023-07-31,354370,Jul
2023-08-31,348450,Aug
2023-09-30,340615,Sep


In [13]:
# Create the Plotly figure with text parameter
fig = px.bar(df_monthly, x='month_name', y='total_sales', template=plotly_template, text='total_sales')

# Set the layout
fig.update_layout(title='Total Sales by Month', xaxis_title='Month', yaxis_title='Total Sales ($)', yaxis_tickprefix='$')

# Show the plot
fig.show()

# Save the chart as a PNG image
fig.write_image(output_dir / 'monthly_sales.png', width=1200, height=400, scale=4)

Total Sales by Product

In [14]:
# Execute the query and load the results into a Pandas dataframe
query = '''
SELECT c.product_name as product_name, SUM(s.total_price) as total_sales
FROM sales s
JOIN products c ON s.product_id = c.product_id
GROUP BY product_name
ORDER BY total_sales DESC
'''
df = pd.read_sql_query(query, conn)
df

,product_name,total_sales
0,Product C,1314800
1,Product A,1096745
2,Product E,904640
3,Product D,566265
4,Product B,326380


In [15]:
# Create the Plotly figure with text
fig = px.bar(df, x='product_name', y='total_sales', template=plotly_template, text='total_sales')

# Set the layout
fig.update_layout(title='Total Sales by Product', xaxis_title='Product', yaxis_title='Total Sales ($)', yaxis_tickprefix='$')

# Show the plot
fig.show()

# Save the chart as a PNG image
fig.write_image(output_dir / 'product_sales.png', width=1200, height=400, scale=4)

Top Employee by Sales

In [16]:
# Execute the query and load results into a Pandas dataframe
query = '''
SELECT c.first_name || ' ' || c.last_name as customer_name, SUM(s.total_price) as total_sales
FROM sales s
JOIN customers c ON s.customer_id = c.customer_id
GROUP by customer_name
ORDER by total_sales DESC
LIMIT 10
'''
df = pd.read_sql_query(query, conn)
df

,customer_name,total_sales
0,Kelly Kapoor,435235
1,Kevin Malone,425205
2,Ryan Howard,423765
3,Oscar Martinez,423425
4,Angela Martin,423170
5,Micheal Scott,422280
6,Toby Flenderson,419585
7,Jim Halpert,415305
8,Pam Beesly,414570
9,Dwight Schrute,406290


In [17]:
# Create the Plotly figure
fig = px.bar(df, x='customer_name', y='total_sales', template=plotly_template, text='total_sales')

# Set the layout
fig.update_layout(title='Top 10 Employees by Sales', xaxis_title='Employee', yaxis_title='Total Sales ($)', yaxis_tickprefix='$')

# Show the plot
fig.show()

# Save the chart as a PNG image
fig.write_image(output_dir / 'employee_sales.png', width=1200, height=400, scale=4)

Create PDF Report

In [18]:
# Define the font color
font_color = (64, 64, 64)

# Find all PNG files in the output folder
chart_filenames = [str(chart_path) for chart_path in output_dir.glob('*.png')]

# Create a PDF and set the page size
pdf = FPDF()
pdf.add_page()
pdf.set_font('Arial', 'B', 24)

# Add the page title
title = f"Sales Report as of {date.today().strftime('%m/%d/%Y')}"
pdf.set_text_color(*font_color)
pdf.cell(0, 20, title, align='C', ln=1)

# Add each chart to the PDF
for chart_filename in chart_filenames:
    pdf.ln(10)
    pdf.image(chart_filename, x=None, y=None, w=pdf.w - 20, h=0)

# Save the PDF to a file
pdf.output(output_dir / "sales_report.pdf", "F")

''